# Loop 1 Analysis: Snapshot Ensemble Strategy

Goal: Score all 105 snapshots and create an ensemble taking the best N from each.
This is the key strategy from the jonathanchan kernel.

In [1]:
import pandas as pd
import numpy as np
import math
import os
import glob
from numba import njit
from tqdm import tqdm

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a single N-tree configuration"""
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c, s = math.cos(r), math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx, mxx = min(mnx, X), max(mxx, X)
            mny, mxy = min(mny, Y), max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    """Remove 's' prefix from values"""
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

print('Functions defined')

Functions defined


In [2]:
# Find all submission CSVs in snapshots
snapshot_dir = '/home/nonroot/snapshots/santa-2025/'
all_csvs = []

for root, dirs, files in os.walk(snapshot_dir):
    for f in files:
        if f.endswith('.csv'):
            all_csvs.append(os.path.join(root, f))

print(f'Found {len(all_csvs)} CSV files in snapshots')

Found 3285 CSV files in snapshots


In [3]:
# Score each CSV and track best per N
best = {n: {'score': 1e300, 'data': None, 'src': None} for n in range(1, 201)}
csv_scores = {}  # Track total score per CSV

for fp in tqdm(all_csvs, desc='Scoring snapshots'):
    try:
        df = pd.read_csv(fp)
    except Exception as e:
        continue
    
    if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
        continue
    
    # Check if it has all N values
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    n_values = df['N'].unique()
    if len(n_values) < 200:
        continue  # Skip incomplete submissions
    
    total_score = 0.0
    for n in range(1, 201):
        g = df[df['N'] == n]
        if len(g) != n:
            continue
        
        xs = strip(g['x'].values)
        ys = strip(g['y'].values)
        ds = strip(g['deg'].values)
        sc = score_group(xs, ys, ds, TX, TY)
        total_score += sc
        
        if sc < best[n]['score']:
            best[n]['score'] = float(sc)
            best[n]['data'] = g.drop(columns=['N']).copy()
            best[n]['src'] = fp
    
    csv_scores[fp] = total_score

print(f'\nScored {len(csv_scores)} complete submissions')

Scoring snapshots:   0%|          | 0/3285 [00:00<?, ?it/s]

Scoring snapshots:   0%|          | 1/3285 [00:00<26:55,  2.03it/s]

Scoring snapshots:   0%|          | 3/3285 [00:00<10:25,  5.24it/s]

Scoring snapshots:   0%|          | 5/3285 [00:00<08:03,  6.78it/s]

Scoring snapshots:   0%|          | 6/3285 [00:00<07:22,  7.41it/s]

...

Scoring snapshots: 100%|█████████▉| 3280/3285 [04:48<00:00, 11.04it/s]

Scoring snapshots: 100%|█████████▉| 3282/3285 [04:48<00:00, 10.22it/s]

Scoring snapshots: 100%|█████████▉| 3284/3285 [04:48<00:00, 10.50it/s]

Scoring snapshots: 100%|██████████| 3285/3285 [04:49<00:00, 11.37it/s]


Scored 3250 complete submissions


In [4]:
# Show top 10 submissions by total score
print('Top 10 submissions by total score:')
for i, (fp, score) in enumerate(sorted(csv_scores.items(), key=lambda x: x[1])[:10]):
    print(f'{i+1}. {score:.6f} - {fp.split("/")[-3]}')

# Calculate ensemble score
ensemble_score = sum(best[n]['score'] for n in range(1, 201))
print(f'\nEnsemble score (best per N): {ensemble_score:.6f}')
print(f'Best single submission: {min(csv_scores.values()):.6f}')
print(f'Improvement from ensemble: {min(csv_scores.values()) - ensemble_score:.6f}')

Top 10 submissions by total score:
1. 27.414787 - code
2. 27.414787 - experiments
3. 27.414787 - experiments
4. 27.478179 - experiments
5. 39.508442 - 21145963314
6. 51.423527 - code
7. 51.663965 - experiments
8. 55.906139 - experiments
9. 67.727119 - code
10. 67.772662 - code

Ensemble score (best per N): 27.414787
Best single submission: 27.414787
Improvement from ensemble: 0.000000


In [5]:
# Override N=1 with optimal value (45 degrees)
manual_data = pd.DataFrame({
    'id': ['001_0'],
    'x': ['s0.0'],
    'y': ['s0.0'],
    'deg': ['s45.0']
})
xs = strip(manual_data['x'].values)
ys = strip(manual_data['y'].values)
ds = strip(manual_data['deg'].values)
sc = score_group(xs, ys, ds, TX, TY)
best[1]['score'] = float(sc)
best[1]['data'] = manual_data.copy()
best[1]['src'] = 'optimal_n1'

print(f'N=1 optimal score: {sc:.6f}')

# Recalculate ensemble score
ensemble_score = sum(best[n]['score'] for n in range(1, 201))
print(f'Ensemble score with optimal N=1: {ensemble_score:.6f}')

N=1 optimal score: 0.661250
Ensemble score with optimal N=1: 27.414787


In [6]:
# Analyze which N values have the most room for improvement
print('\nN values with highest scores (most room for improvement):')
n_scores = [(n, best[n]['score']) for n in range(1, 201)]
n_scores_sorted = sorted(n_scores, key=lambda x: x[1], reverse=True)

for n, score in n_scores_sorted[:20]:
    print(f'  N={n:3d}: {score:.6f}')


N values with highest scores (most room for improvement):
  N=  1: 0.661250
  N=  2: 0.338427
  N=  3: 0.262632
  N= 57: 0.231956
  N= 74: 0.221013
  N= 75: 0.214885
  N=  5: 0.206839
  N= 76: 0.202441
  N=  4: 0.199376
  N= 56: 0.196873
  N= 86: 0.191829
  N= 65: 0.190942
  N= 73: 0.186112
  N= 84: 0.184652
  N= 61: 0.184133
  N= 97: 0.180768
  N= 60: 0.180633
  N= 66: 0.179472
  N= 81: 0.179201
  N=101: 0.179159


In [7]:
# Count how many N values each source contributes
source_counts = {}
for n in range(1, 201):
    src = best[n]['src']
    if src:
        source_counts[src] = source_counts.get(src, 0) + 1

print('\nSources contributing to ensemble:')
for src, count in sorted(source_counts.items(), key=lambda x: -x[1])[:10]:
    print(f'  {count:3d} N values from {src.split("/")[-3] if "/" in src else src}')


Sources contributing to ensemble:
  199 N values from code
    1 N values from optimal_n1


In [8]:
# Create ensemble submission
rows = []
for n in range(1, 201):
    if best[n]['data'] is not None:
        rows.append(best[n]['data'])

ensemble_df = pd.concat(rows, ignore_index=True)

# Sort by N and index
ensemble_df['sn'] = ensemble_df['id'].str.split('_').str[0].astype(int)
ensemble_df['si'] = ensemble_df['id'].str.split('_').str[1].astype(int)
ensemble_df = ensemble_df.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
ensemble_df = ensemble_df[['id', 'x', 'y', 'deg']]

print(f'Ensemble submission has {len(ensemble_df)} rows')
print(ensemble_df.head())

Ensemble submission has 20100 rows
      id                    x                    y                   deg
0  001_0                 s0.0                 s0.0                 s45.0
1  002_0  s-0.031131500992148  s-0.181577662297216  s223.425025586567472
2  002_1   s0.264851133123133  s-0.513466753944634   s44.194648564006400
3  003_0   s0.961600534826014   s1.029104871637799  s130.886212151358791
4  003_1   s0.977972198049508   s0.794759394682743   s55.961368140998331


In [9]:
# Save ensemble submission
os.makedirs('/home/submission', exist_ok=True)
ensemble_df.to_csv('/home/submission/submission.csv', index=False)
print('Saved ensemble to /home/submission/submission.csv')

# Verify score
verify_df = pd.read_csv('/home/submission/submission.csv')
verify_df['N'] = verify_df['id'].str.split('_').str[0].astype(int)
verify_score = 0.0
for n in range(1, 201):
    g = verify_df[verify_df['N'] == n]
    xs = strip(g['x'].values)
    ys = strip(g['y'].values)
    ds = strip(g['deg'].values)
    verify_score += score_group(xs, ys, ds, TX, TY)

print(f'Verified ensemble score: {verify_score:.6f}')
print(f'Target: 68.894234')
print(f'Gap: {verify_score - 68.894234:.6f}')

Saved ensemble to /home/submission/submission.csv


Verified ensemble score: 27.414787
Target: 68.894234
Gap: -41.479447


In [10]:
# This seems too good - let's verify the submission is valid
# Check if coordinates are in valid range and format is correct

print("Checking submission validity...")
verify_df = pd.read_csv('/home/submission/submission.csv')

# Check format
print(f"Columns: {verify_df.columns.tolist()}")
print(f"Total rows: {len(verify_df)}")

# Check if all N values are present
verify_df['N'] = verify_df['id'].str.split('_').str[0].astype(int)
n_counts = verify_df.groupby('N').size()
print(f"\nN values present: {len(n_counts)}")
print(f"All N have correct count: {all(n_counts == n_counts.index)}")

# Check coordinate ranges
xs = verify_df['x'].str[1:].astype(float)
ys = verify_df['y'].str[1:].astype(float)
print(f"\nX range: [{xs.min():.6f}, {xs.max():.6f}]")
print(f"Y range: [{ys.min():.6f}, {ys.max():.6f}]")
print(f"Coordinates in [-100, 100]: {xs.min() >= -100 and xs.max() <= 100 and ys.min() >= -100 and ys.max() <= 100}")

Checking submission validity...
Columns: ['id', 'x', 'y', 'deg']
Total rows: 20100

N values present: 200
All N have correct count: True

X range: [-2.164479, 3.896870]
Y range: [-2.454636, 4.102272]
Coordinates in [-100, 100]: True


In [11]:
# Find the source of this amazing solution
print("Source of best solution:")
for src, count in sorted(source_counts.items(), key=lambda x: -x[1])[:5]:
    print(f"  {count:3d} N values from {src}")

Source of best solution:
  199 N values from /home/nonroot/snapshots/santa-2025/21322576451/code/experiments/ensemble_best.csv
    1 N values from optimal_n1


In [13]:
# Let's look at the top submissions more carefully
print("Top 20 submissions by total score:")
for i, (fp, score) in enumerate(sorted(csv_scores.items(), key=lambda x: x[1])[:20]):
    print(f'{i+1:2d}. {score:.6f} - {fp}')

Top 20 submissions by total score:
 1. 27.414787 - /home/nonroot/snapshots/santa-2025/21322576451/code/experiments/ensemble_best.csv
 2. 27.414787 - /home/nonroot/snapshots/santa-2025/21328309666/code/experiments/002_ensemble/ensemble_valid.csv
 3. 27.414787 - /home/nonroot/snapshots/santa-2025/21328309666/code/experiments/002_ensemble/ensemble.csv
 4. 27.478179 - /home/nonroot/snapshots/santa-2025/21165878844/code/experiments/002_ensemble/submission.csv
 5. 39.508442 - /home/nonroot/snapshots/santa-2025/21145963314/code/submission.csv
 6. 51.423527 - /home/nonroot/snapshots/santa-2025/21145968755/code/submission_candidates/candidate_004.csv
 7. 51.663965 - /home/nonroot/snapshots/santa-2025/21117525284/code/experiments/004_jonathanchan_optimizer/submission_opt.csv
 8. 55.906139 - /home/nonroot/snapshots/santa-2025/21145963314/code/experiments/011_comprehensive_ensemble/ensemble.csv
 9. 67.727119 - /home/nonroot/snapshots/santa-2025/21108486172/code/experiments/submission_v21.csv
10. 6

In [ ]:
# Check for overlaps in the ensemble submission using Shapely
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate

# Create base tree polygon
base_tree = Polygon(zip(TX, TY))
print(f'Base tree area: {base_tree.area:.6f}')
print(f'Base tree is valid: {base_tree.is_valid}')

def create_tree_polygon(x, y, deg):
    """Create a tree polygon at position (x, y) with rotation deg"""
    tree = rotate(base_tree, deg, origin=(0, 0))
    tree = translate(tree, x, y)
    return tree

def check_overlaps(df, n):
    """Check for overlaps in a specific N-tree configuration"""
    mask = df['id'].str.startswith(f'{n:03d}_')
    group = df[mask]
    
    if len(group) != n:
        return False, f'Wrong count: {len(group)} vs {n}'
    
    # Parse coordinates
    xs = group['x'].str[1:].astype(float).values
    ys = group['y'].str[1:].astype(float).values
    degs = group['deg'].str[1:].astype(float).values
    
    # Create polygons
    polygons = [create_tree_polygon(xs[i], ys[i], degs[i]) for i in range(n)]
    
    # Check all pairs for overlap
    for i in range(n):
        for j in range(i+1, n):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                # Check if intersection has area (not just touching)
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > 1e-12:  # Small tolerance
                    return False, f'Trees {i} and {j} overlap with area {intersection.area:.2e}'
    
    return True, 'No overlaps'

print('Overlap checking functions defined')

In [ ]:
# Check group 008 specifically (the one that failed)
ensemble_df = pd.read_csv('/home/submission/submission.csv')
print('Checking group 008 for overlaps...')
valid, msg = check_overlaps(ensemble_df, 8)
print(f'Group 008: valid={valid}, message={msg}')

In [ ]:
# Check ALL groups for overlaps
print('Checking all groups for overlaps...')
invalid_groups = []
for n in range(1, 201):
    valid, msg = check_overlaps(ensemble_df, n)
    if not valid:
        invalid_groups.append((n, msg))
        print(f'  N={n}: INVALID - {msg}')

if not invalid_groups:
    print('\nAll groups are valid! No overlaps detected.')
else:
    print(f'\nFound {len(invalid_groups)} invalid groups')